In [1]:
import numpy as np
import gym
import tensorflow as tf

# gamespace 
env = gym.make("Pong-v0") # environment info
env.unwrapped.get_action_meanings()

[2017-11-03 08:37:12,071] Making new env: Pong-v0


['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']

In [4]:
n_obs = 80 * 80
n_classes = 4  # ['NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE']
learning_rate = 1e-3
gamma = .99
decay = 0.99              # decay rate for RMSProp gradients

# tf operations
def tf_discount_rewards(tf_r):
    discount_f = lambda a, v: a*gamma + v;
    tf_r_reverse = tf.scan(discount_f, tf.reverse(tf_r,[True, False]))
    tf_discounted_r = tf.reverse(tf_r_reverse,[True, False])
    return tf_discounted_r

# downsampling
def prepro(I):
    """ prepro 210x160x3 uint8 frame into 6400 (80x80) 1D float vector """
    I = I[35:195] # crop
    I = I[::2,::2,0] # downsample by factor of 2
    I[I == 144] = 0  # erase background (background type 1)
    I[I == 109] = 0  # erase background (background type 2)
    I[I != 0] = 1    # everything else (paddles, ball) just set to 1
    return I.astype(np.float32).ravel()

In [7]:
tf.reset_default_graph()

X_raw = tf.placeholder(tf.float64, [None, 6400], name="input")
X_ = tf.cast(tf.reshape(X_raw, (-1, 80, 80, 1)), tf.float32)
Y_ = tf.placeholder(tf.float32, [None, n_classes], name="action")

# tf reward processing (need tf_discounted_epr for policy gradient wizardry)
tf_epr = tf.placeholder(dtype=tf.float32, shape=[None,1], name="tf_epr")
tf_discounted_epr = tf_discount_rewards(tf_epr)
tf_mean, tf_variance= tf.nn.moments(tf_discounted_epr, [0], shift=None, name="reward_moments")
tf_discounted_epr -= tf_mean
tf_discounted_epr /= tf.sqrt(tf_variance + 1e-6)


K = 4  # first convolutional layer output depth
L = 8  # second convolutional layer output depth
M = 12  # third convolutional layer
N = 200  # fully connected layer

# 3x3 patch, 1 input channel (because input image has 1 channel), K output channels
W1 = tf.Variable(tf.truncated_normal([3, 3, 1, K], stddev=0.1))  
B1 = tf.Variable(tf.ones([K])/10)

# 3x3 patch, K input channel, L output channels
W2 = tf.Variable(tf.truncated_normal([3, 3, K, L], stddev=0.1))
B2 = tf.Variable(tf.ones([L])/10)

# 3x3 patch, L input channel, M output channels
W3 = tf.Variable(tf.truncated_normal([3, 3, L, M], stddev=0.1))
B3 = tf.Variable(tf.ones([M])/10)

# w4 is 20x20xM bc Y3 output is 20x20 with M channels
W4 = tf.Variable(tf.truncated_normal([20 * 20 * M, N], stddev=0.1))
B4 = tf.Variable(tf.ones([N])/10)
W5 = tf.Variable(tf.truncated_normal([N, n_classes], stddev=0.1))
B5 = tf.Variable(tf.ones([n_classes])/n_classes)

stride = 1  
Y1 = tf.nn.relu(tf.nn.conv2d(X_, W1, strides=[1, stride, stride, 1], padding='SAME') + B1)  # output is 80x80, stride refers to pixels skipped per convolution

stride = 2
Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W2, strides=[1, stride, stride, 1], padding='SAME') + B2)  # output is 40x40, or 80 / 2
stride = 2
Y3 = tf.nn.relu(tf.nn.conv2d(Y2, W3, strides=[1, stride, stride, 1], padding='SAME') + B3)  # output is 20x20 or 80 / 2

# since Y3 is 20x20 with M channels, we want to reshape Y3 to YY of shape (-1, 20 * 20 * M)
# reshape the output from the third convolution for the fully connected layer
YY = tf.reshape(Y3, shape=[-1, 20 * 20 * M])

# now that this is a flat 1d vector, we can feed it to a vanilla function
z4 = tf.matmul(YY, W4) + B4
Y4 = tf.nn.relu(z4)
Ylogits = tf.matmul(Y4, W5) + B5
Y = tf.nn.softmax(Ylogits)

loss = tf.losses.log_loss(labels=Y_, predictions=Y, weights=tf_discounted_epr)
op = tf.train.AdamOptimizer(0.001).minimize(loss)

In [9]:
observation = env.reset()
prev_x = None
xs,rs,ys = [],[],[]
running_reward = None
reward_sum = 0
episode_number = 0

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    while True:
        env.render()

        cur_x = prepro(observation)
        x = cur_x - prev_x if prev_x is not None else np.zeros(n_obs)
        prev_x = cur_x

        # stochastically sample a policy from the network
        aprob = sess.run(Y, feed_dict={X_raw: np.reshape(x, (1,-1))})[0]
        print ('aprop {}'.format(aprob))
        action = np.random.choice(n_classes, p=aprob)
        label = np.zeros_like(aprob)
        label[action] = 1

        # step the environment and get new measurements
        observation, reward, done, info = env.step(action)
        reward_sum += reward

        # record game history
        xs.append(x)
        ys.append(label)
        rs.append(reward)
        
        if done:
            # update running reward
            running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01

            # parameter update
            rs_float = np.array(rs).astype("float32")
            sess.run(op, feed_dict={X_raw: np.vstack(xs), tf_epr: np.vstack(rs_float), Y_: np.vstack(ys)})

            # print progress console
            if episode_number % 10 == 0:
                print ('ep {}: reward: {}, mean reward: {:3f}'.format(episode_number, reward_sum, running_reward))
            else:
                print ('\tep {}: reward: {}'.format(episode_number, reward_sum))
                
                
            # bookkeeping
            xs,rs,ys = [],[],[] # reset game history
            episode_number += 1 # the Next Episode
            observation = env.reset() # reset env
            reward_sum = 0

aprop [ 0.13199246  0.54077578  0.06347727  0.26375446]
aprop [ 0.13217978  0.54122567  0.06354579  0.26304877]
aprop [ 0.13199246  0.54077578  0.06347727  0.26375446]
aprop [ 0.13199246  0.54077578  0.06347727  0.26375446]
aprop [ 0.13007568  0.54674613  0.06292833  0.26024982]
aprop [ 0.13305962  0.54148984  0.06604157  0.25940901]
aprop [ 0.12915997  0.54249805  0.06312285  0.26521912]
aprop [ 0.13465205  0.53801918  0.06385528  0.26347357]
aprop [ 0.12669744  0.55173743  0.0629821   0.25858304]
aprop [ 0.13294835  0.53856879  0.06261925  0.26586363]
aprop [ 0.13199246  0.54077578  0.06347727  0.26375446]
aprop [ 0.13199246  0.54077578  0.06347727  0.26375446]
aprop [ 0.13372426  0.53525102  0.06425752  0.26676717]
aprop [ 0.13532759  0.54579556  0.06397921  0.25489756]
aprop [ 0.12959655  0.54079574  0.06239508  0.26721272]
aprop [ 0.13138671  0.54019171  0.06351828  0.26490337]
aprop [ 0.13199246  0.54077578  0.06347727  0.26375446]
aprop [ 0.13199246  0.54077578  0.06347727  0.26

aprop [ 0.1281762   0.53881747  0.06262796  0.27037835]
aprop [ 0.13207467  0.54003584  0.06511936  0.26277024]
aprop [ 0.12647752  0.5483349   0.06274968  0.26243791]
aprop [ 0.13530597  0.53255761  0.06608278  0.26605371]
aprop [ 0.1321509   0.54307663  0.0629621   0.26181036]
aprop [ 0.13497603  0.53744489  0.06418996  0.26338914]
aprop [ 0.13476916  0.54320914  0.06268299  0.25933877]
aprop [ 0.13032335  0.54306471  0.06320619  0.26340577]
aprop [ 0.13205771  0.54134989  0.06393465  0.26265773]
aprop [ 0.1312429   0.53811574  0.06399956  0.26664171]
aprop [ 0.12983574  0.54107344  0.06316104  0.26592976]
aprop [ 0.12873341  0.54218835  0.06522976  0.26384851]
aprop [ 0.1322176   0.54285109  0.06221047  0.26272082]
aprop [ 0.13199955  0.53956622  0.06349363  0.26494056]
aprop [ 0.13510734  0.54058564  0.06085522  0.26345181]
aprop [ 0.13147013  0.54258931  0.06278448  0.26315603]
aprop [ 0.13346227  0.53740442  0.06384186  0.26529142]
aprop [ 0.13369654  0.54056442  0.0632792   0.26

aprop [ 0.13307559  0.54200768  0.06457224  0.26034448]
aprop [ 0.12795286  0.54131049  0.06341169  0.26732492]
aprop [ 0.13311923  0.53920108  0.06311727  0.26456246]
aprop [ 0.13014176  0.54270196  0.06328154  0.26387474]
aprop [ 0.12953818  0.54311967  0.06462625  0.26271594]
aprop [ 0.13025975  0.5434801   0.06445562  0.26180452]
aprop [ 0.13240071  0.54267693  0.06195334  0.26296908]
aprop [ 0.13479692  0.53338844  0.06312279  0.26869187]
aprop [ 0.13217317  0.54196912  0.06242775  0.26342988]
aprop [ 0.1345154   0.54164207  0.06208719  0.26175538]
aprop [ 0.13390654  0.53728831  0.06424364  0.26456153]
aprop [ 0.13373965  0.53864312  0.06393386  0.26368341]
aprop [ 0.13181789  0.54539037  0.0639049   0.25888681]
aprop [ 0.1326573   0.54159349  0.06325761  0.26249161]
aprop [ 0.13132487  0.54240918  0.06402166  0.26224431]
aprop [ 0.13030517  0.54101014  0.06358382  0.26510087]
aprop [ 0.13085712  0.54207349  0.06155918  0.26551017]
aprop [ 0.13068154  0.54323953  0.06321566  0.26

aprop [ 0.13109834  0.53878736  0.06426565  0.26584861]
aprop [ 0.13369654  0.54056442  0.0632792   0.26245987]
aprop [ 0.13222572  0.54250854  0.0645361   0.26072964]
aprop [ 0.13195071  0.5452736   0.06312191  0.25965375]
aprop [ 0.13095246  0.5451476   0.06383741  0.26006261]
aprop [ 0.13044237  0.54017824  0.06392911  0.2654503 ]
aprop [ 0.13042001  0.54160416  0.06106323  0.26691258]
aprop [ 0.13056193  0.54312849  0.06367837  0.26263124]
aprop [ 0.13102876  0.53911448  0.06296419  0.26689264]
aprop [ 0.13366914  0.5346266   0.06391359  0.26779068]
aprop [ 0.13021098  0.54348481  0.06287919  0.2634249 ]
aprop [ 0.13303824  0.53876299  0.06377     0.26442879]
aprop [ 0.13170941  0.54065394  0.06330172  0.26433489]
aprop [ 0.13245946  0.54281825  0.06350323  0.26121905]
aprop [ 0.13472164  0.54463816  0.06471121  0.25592905]
aprop [ 0.13121434  0.54207927  0.06239017  0.26431623]
aprop [ 0.13272417  0.53095579  0.06266485  0.27365515]
aprop [ 0.12803331  0.54782885  0.06208078  0.26

aprop [ 0.13476586  0.54326636  0.06468678  0.25728104]
aprop [ 0.13193023  0.53777283  0.0612139   0.26908302]
aprop [ 0.13242951  0.53567988  0.06365736  0.26823315]
aprop [ 0.12662841  0.54781908  0.06301191  0.26254055]
aprop [ 0.13196602  0.5410229   0.06537381  0.26163724]
aprop [ 0.12952852  0.54333413  0.06424364  0.26289371]
aprop [ 0.13347377  0.53667873  0.06459264  0.26525491]
aprop [ 0.13506678  0.54232734  0.06272066  0.2598851 ]
aprop [ 0.13290963  0.54183841  0.06486959  0.26038244]
aprop [ 0.1306048   0.53878188  0.06399215  0.2666212 ]
aprop [ 0.13037463  0.54337233  0.06301947  0.26323351]
aprop [ 0.12937151  0.54626918  0.06190675  0.26245251]
aprop [ 0.13343057  0.53537536  0.06367047  0.26752362]
aprop [ 0.13134389  0.538059    0.06451691  0.26608011]
aprop [ 0.13115264  0.54013467  0.06330046  0.26541224]
aprop [ 0.13251264  0.54575557  0.06305664  0.25867516]
aprop [ 0.13096048  0.54189658  0.06317949  0.26396349]
aprop [ 0.12948638  0.53768325  0.06406721  0.26

...

aprop [  6.81340173e-02   8.70773613e-01   6.03095926e-02   7.82739371e-04]
aprop [  6.93065673e-02   8.67463529e-01   6.24691658e-02   7.60809169e-04]
aprop [  6.58360869e-02   8.75120342e-01   5.83107322e-02   7.32723973e-04]
aprop [  6.92998320e-02   8.67312193e-01   6.26146123e-02   7.73401116e-04]
aprop [  7.00148642e-02   8.69245529e-01   5.99818267e-02   7.57738890e-04]
aprop [  6.91021830e-02   8.68714631e-01   6.14312477e-02   7.51899090e-04]
aprop [  6.95971027e-02   8.68584454e-01   6.10510930e-02   7.67372199e-04]
aprop [  6.88316971e-02   8.69504333e-01   6.09034672e-02   7.60532799e-04]
aprop [  6.85728565e-02   8.70121598e-01   6.05399795e-02   7.65580218e-04]
aprop [  6.83159754e-02   8.71428728e-01   5.94873726e-02   7.67874066e-04]
aprop [  6.83909133e-02   8.69369030e-01   6.14928864e-02   7.47165293e-04]
aprop [  6.84466958e-02   8.71979117e-01   5.88117950e-02   7.62449403e-04]
aprop [  7.00247809e-02   8.68005753e-01   6.12022579e-02   7.67271675e-04]
aprop [  6.9

aprop [  6.86356947e-02   8.70319903e-01   6.02838881e-02   7.60469411e-04]
aprop [  6.85275868e-02   8.71205688e-01   5.95092252e-02   7.57380796e-04]
aprop [  6.76754192e-02   8.73134315e-01   5.84359355e-02   7.54340494e-04]
aprop [  6.77515119e-02   8.72131824e-01   5.93649931e-02   7.51677901e-04]
aprop [  6.84762746e-02   8.70788097e-01   5.99621311e-02   7.73579930e-04]
aprop [  6.91628754e-02   8.68867874e-01   6.12076260e-02   7.61608593e-04]
aprop [  6.83604181e-02   8.71098757e-01   5.97860068e-02   7.54793524e-04]
aprop [  6.90931678e-02   8.69711578e-01   6.04312941e-02   7.64030556e-04]
aprop [  6.87132031e-02   8.69620025e-01   6.09159246e-02   7.50830106e-04]
aprop [  7.04127997e-02   8.66486728e-01   6.23490810e-02   7.51348794e-04]
aprop [  6.91813156e-02   8.69959116e-01   6.01055995e-02   7.53960980e-04]
aprop [  6.79893643e-02   8.71576130e-01   5.96770346e-02   7.57505244e-04]
aprop [  7.06908628e-02   8.67823720e-01   6.06867597e-02   7.98639492e-04]
aprop [  6.8

aprop [  6.80477470e-02   8.71845782e-01   5.93371503e-02   7.69256963e-04]
aprop [  6.71240762e-02   8.72044921e-01   6.00854009e-02   7.45642348e-04]
aprop [  6.76868930e-02   8.73672187e-01   5.79017401e-02   7.39179901e-04]
aprop [  6.94182962e-02   8.68556201e-01   6.12559356e-02   7.69652950e-04]
aprop [  6.95977584e-02   8.68611813e-01   6.10203259e-02   7.70230545e-04]
aprop [  6.81736916e-02   8.71104538e-01   5.99532425e-02   7.68502534e-04]
aprop [  6.91452473e-02   8.70310724e-01   5.98007627e-02   7.43213401e-04]
aprop [  6.89335316e-02   8.69905293e-01   6.03955425e-02   7.65638892e-04]
aprop [  6.86637536e-02   8.70668650e-01   5.99038750e-02   7.63709832e-04]
aprop [  6.87033087e-02   8.68634939e-01   6.18805811e-02   7.81140523e-04]
aprop [  6.55728281e-02   8.75476718e-01   5.82250394e-02   7.25443067e-04]
aprop [  6.94058612e-02   8.68088126e-01   6.17322065e-02   7.73890002e-04]
aprop [  6.92943037e-02   8.69204998e-01   6.07397929e-02   7.60850147e-04]
aprop [  7.0

aprop [  6.77968785e-02   8.71077240e-01   6.03665300e-02   7.59365328e-04]
aprop [  6.84210137e-02   8.70586455e-01   6.02349341e-02   7.57558504e-04]
aprop [  6.87683746e-02   8.70628357e-01   5.98449707e-02   7.58285285e-04]
aprop [  6.96120784e-02   8.68931592e-01   6.06926046e-02   7.63697375e-04]
aprop [  6.99569210e-02   8.71100605e-01   5.81815504e-02   7.60973024e-04]
aprop [  6.91209510e-02   8.69800627e-01   6.03126101e-02   7.65871373e-04]
aprop [  7.06383362e-02   8.66465449e-01   6.21300526e-02   7.66160025e-04]
aprop [  6.96781427e-02   8.68831515e-01   6.07218444e-02   7.68477097e-04]
aprop [  6.85354248e-02   8.70617509e-01   6.01096265e-02   7.37375580e-04]
aprop [  6.83660209e-02   8.70600820e-01   6.02786206e-02   7.54535431e-04]
aprop [  6.85730428e-02   8.70588720e-01   6.00860305e-02   7.52237334e-04]
aprop [  6.77856579e-02   8.72336626e-01   5.91210574e-02   7.56596331e-04]
aprop [  6.72243983e-02   8.72541249e-01   5.94718754e-02   7.62485957e-04]
aprop [  6.9

KeyboardInterrupt: 